In [44]:
# 必要なライブラリをインストール
! pip install pandas
! pip install openai faiss-cpu transformers
! pip install faiss-cpu
! pip install torch
! pip install numpy
! pip install torch

In [2]:
! pip install -qU langchain langchain-community langchain_text_splitters langchain_chroma
! pip install -qU langchain-openai

### Part1. Load from the cloud on the Kaggle database

Kaggle is the famous platform which is offering machine learning compititions and opensource datasets

Note that if you want to use Kaggle, you must have an account but one can easily log in using one's Google or Fakebook or Yahoo account

We need to install Kaggle API (Application Programming Interface) first:

In [1]:
! pip install kaggle

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105797 sha256=806d80a5e7a85bff173dd323dd57722f7b423ad2895e93f82d03411925a4c855
  Stored in directory: c:\users\wangs\appdata\local\pip\cache\wheels\2b\af\a9\70bffa2773af622d2ebea9c8d407720b86e67bd40c465bf837
Successfully built kaggle


On the Kaggle home page in your browser, click on the icon in the upper right corner and select "Account" from the menu bar to go to the account management page. In the "API" section, select "Create New API Token" to download the kaggle.json file to your computer. **After uploading that json file to colab by clicking on the folder symbol in the leftmost tab**, run the following cell.

In [16]:
path = "C:/Users/wangs/.kaggle"
os.makedirs(path, exist_ok=True)
print(f"指定したディレクトリにAPIファイルを入れてください:{path}")

指定したディレクトリにAPIファイルを入れてください:C:/Users/wangs/.kaggle


In [4]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.model_list_cli()

Next Page Token = CfDJ8GrUjsAKvhxNhvm67MmtKLl39PI19c56I5tY-a_zC4a4Hv2AGugR0W3XR5n4ElQhQ8XVOEzjSsXWZpCTF1zKo5Q
    id  ref                                                       title                                          subtitle                                                                                                                                                                                                              author              
------  --------------------------------------------------------  ---------------------------------------------  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ------------------  
121027  metaresearch/llama-3.2                                    Llama 3.2                                      The Meta Llama 3.2 collection of multilingual large language models (LLMs) is a c

In [5]:
! kaggle competitions list

ref                                                                                deadline             category                reward  teamCount  userHasEntered  
---------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2      2025-04-01 23:59:00  Featured         2,117,152 Usd        412           False  
https://www.kaggle.com/competitions/gemma-language-tuning                          2025-01-15 00:59:00  Analytics          150,000 Usd          0           False  
https://www.kaggle.com/competitions/jane-street-real-time-market-data-forecasting  2025-01-13 23:59:00  Featured           120,000 Usd       1327           False  
https://www.kaggle.com/competitions/gemini-long-context                            2024-12-01 23:59:00  Analytics          100,000 Usd          0           False  
https://www.kagg

we can download datasets by using following command
~~~
! kaggle datasets download <name-of-dataset>
~~~


In [12]:
! kaggle datasets download noxmoon/chinese-official-daily-news-since-2016

Dataset URL: https://www.kaggle.com/datasets/noxmoon/chinese-official-daily-news-since-2016
License(s): unknown




  0%|          | 0.00/8.43M [00:00<?, ?B/s]
 12%|█▏        | 1.00M/8.43M [00:00<00:05, 1.34MB/s]
 24%|██▎       | 2.00M/8.43M [00:00<00:02, 2.42MB/s]
 36%|███▌      | 3.00M/8.43M [00:01<00:01, 3.05MB/s]
 47%|████▋     | 4.00M/8.43M [00:02<00:02, 1.68MB/s]
 59%|█████▉    | 5.00M/8.43M [00:02<00:01, 1.93MB/s]
 71%|███████   | 6.00M/8.43M [00:03<00:01, 2.09MB/s]
 83%|████████▎ | 7.00M/8.43M [00:03<00:00, 2.19MB/s]
 95%|█████████▍| 8.00M/8.43M [00:04<00:00, 2.18MB/s]
100%|██████████| 8.43M/8.43M [00:04<00:00, 2.29MB/s]
100%|██████████| 8.43M/8.43M [00:04<00:00, 2.13MB/s]


In [13]:
import zipfile

# ZIPファイルのパスを指定
zip_file_path = "chinese-official-daily-news-since-2016.zip"
extract_path = "./extracted_data"  # 解凍先ディレクトリを指定

# 解凍先のディレクトリが存在しない場合は作成
os.makedirs(extract_path, exist_ok=True)

# ZIPファイルを解凍
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"{zip_file_path}を{extract_path}に解凍しました。")


chinese-official-daily-news-since-2016.zipを./extracted_dataに解凍しました。


### part2 make a RAG using ChatGPT API

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
import pandas as pd

# CSVファイルの読み込み
file_path = 'extracted_data/chinese_news.csv'
data = pd.read_csv(file_path)

# 最初の数行を確認
data.head()

,date,tag,headline,content
0,2016-01-01,详细全文,陆军领导机构火箭军战略支援部队成立大会在京举行 习近平向中国人民解放军陆军火箭军战略支援部队...,中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...
1,2016-01-01,详细全文,中央军委印发《关于深化国防和军队改革的意见》,经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。\n《意见》强...
2,2016-01-01,详细全文,《习近平关于严明党的纪律和规矩论述摘编》出版发行,由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...
3,2016-01-01,详细全文,以实际行动向党中央看齐 向高标准努力,广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...
4,2016-01-01,详细全文,【年终特稿】关键之年 改革挺进深水区,刚刚过去的2015年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，...


データには以下の列があります：

- date: ニュースの日付
- tag: ニュースのタグ
- headline: ニュースの見出し
- content: ニュースの内容（この列を主に使用）

次に、content列を使用してドキュメントのベクトル化とインデックス化を行い、RAGモデルの検索エンジン部分を作成します。faissを使ってインデックス化し、transformersを使ってテキストをベクトル化します。 

In [4]:
import faiss
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# モデルとトークナイザーの準備（Sentence Transformersを使用）
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# ベクトル化の関数
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings[0].numpy()

# サンプルデータの取得
sample_data = data['headline'].sample(5000, random_state=1)  # ランダムに10件を抽出

# サンプルデータでベクトル化とインデックス化を試行
try:
    sample_embeddings = np.array([embed_text(text) for text in sample_data])
    sample_index = faiss.IndexFlatL2(sample_embeddings.shape[1])
    sample_index.add(sample_embeddings)
    print("サンプルインデックスが正常に作成されました。")
except Exception as e:
    print("エラーが発生しました:", e)


c:\Users\wangs\anaconda3\envs\DH\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


サンプルインデックスが正常に作成されました。


In [6]:
def retrieve_documents(query, k=3):
    # 質問をベクトル化
    query_vector = embed_text(query).reshape(1, -1)
    distances, indices = sample_index.search(query_vector, k)  # kは取得するドキュメント数
    return [sample_data.iloc[i] for i in indices[0]]


In [ ]:
from openai import OpenAI

api_key = ""
client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

def generate_answer(query, retrieved_docs):
    context = "\n".join(retrieved_docs)
    prompt = f"以下のニュース情報に基づいて質問に答えてください。\n\n{context}\n\n質問: {query}\n答え:"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1500
    )
    return response.choices[0].message.content


In [28]:
# 質問を設定し、検索と生成を行う
query = "2016-01-01のニュースをまとめてください"
retrieved_docs = retrieve_documents(query)
answer = generate_answer(query, retrieved_docs)

print("生成された答え:", answer)

生成された答え: 2016年1月1日のニュースによれば、北京市は「十三五」规划を発表し、2020年までに北京城六区の常住人口を2014年の基準から約15%減少させることを目指しています。この対策には非首都機能の調整と人口の最適化が含まれています。

また、2015年末までに全国で783万套の保障性安居工程が開工し、772万套が基本的に完成しました。これは2015年度の目標を上回る達成です。特に棚戸区改造に関しては601万套が開工し、年度


In [13]:
# 質問を設定し、検索と生成を行う
query = "北京ダックのニュースをまとめてください"
retrieved_docs = retrieve_documents(query)
answer = generate_answer(query, retrieved_docs)

print("生成された答え:", answer)

生成された答え: 提供された情報には北京ダックに関するニュースは含まれていません。代わりに、以下のニュースが報告されています：

1. 北京の鉄道警察が偽チケットの製造および販売拠点を捜索し、壊滅させました。
2. 习近平（シー・チンピング）国家主席が北京市の八一学校を訪問し、教育方針の全面的な実施を強調し、中国の基礎教育を向上させる努力を呼びかけました。
3. 北京市は非首都機能の解消を進めており、産業の増減が報告されています。

北京ダックに関する情報が必要であれば、別のソースを参照する必要があります。


次に、情報検索を実行し、ユーザーの質問に関連するニュース記事を検索する仕組みを構築します。質問から類似の文書を検索して応答を生成します。

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors

In [ ]:
# TF-IDFベクトル化を再設定：次元数を削減
vectorizer = TfidfVectorizer(max_features=1000)  # 次元を5000 -> 1000に削減
tfidf_matrix = vectorizer.fit_transform(data)

# k近傍法を使ったスパース行列でのインデックス作成
knn_index = NearestNeighbors(metric='cosine', algorithm='brute')
knn_index.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [76]:
# インデックスに登録されたサンプル数を確認
registered_samples = tfidf_matrix.shape[0]

# 検索クエリを実行する際に、登録サンプル数以下のn_neighborsを設定
def search_news_fixed(query, k=5):
    # kを登録サンプル数以下に調整
    k = min(k, registered_samples)
    # 質問をTF-IDFでベクトル化
    query_vector = vectorizer.transform([query])
    # 最近傍検索
    distances, indices = knn_index.kneighbors(query_vector, n_neighbors=k)
    # 検索結果を返す
    results = []
    for idx in indices[0]:
        results.append({
            "headline": data.iloc[idx]["headline"],
            "content": data.iloc[idx]["content"]
        })
    return results

# 修正版で検索の例
example_query = "国防と軍隊の改革"
search_results_fixed = search_news_fixed(example_query)

# 修正版の検索結果を表示
search_results_fixed


[{'headline': '陆军领导机构火箭军战略支援部队成立大会在京举行 习近平向中国人民解放军陆军火箭军战略支援部队授予军旗并致训词',
  'content': '中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015年12月31日在八一大楼隆重举行。中共中央总书记、国家主席、中央军委主席习近平向陆军、火箭军、战略支援部队授予军旗并致训词，代表党中央和中央军委向同志们、向全军部队致以热烈祝贺，强调要坚持以党在新形势下的强军目标为引领，深入贯彻新形势下军事战略方针，全面实施改革强军战略，坚定不移走中国特色强军之路，时刻听从党和人民召唤，忠实履行党和人民赋予的神圣使命，为实现中国梦强军梦作出新的更大的贡献。\n下午4时，成立大会开始，全场高唱国歌。仪仗礼兵护卫着鲜艳军旗，正步行进到主席台前。习近平将军旗郑重授予陆军司令员李作成、政治委员刘雷，火箭军司令员魏凤和政治委员王家胜，战略支援部队司令员高津、政治委员刘福连。陆军、火箭军、战略支援部队主要领导，军容严整、精神抖擞，向习近平敬礼，从习近平手中接过军旗。全场官兵向军旗敬礼。\n授旗仪式后，习近平致训词。他指出：“成立陆军领导机构、火箭军、战略支援部队，是党中央和中央军委着眼实现中国梦强军梦作出的重大决策，是构建中国特色现代军事力量体系的战略举措，必将成为我军现代化建设的一个重要里程碑，载入人民军队史册。”\n习近平强调，陆军是党最早建立和领导的武装力量，历史悠久，敢打善战，战功卓著，为党和人民建立了不朽功勋。陆军对维护国家主权、安全和发展利益具有不可替代的作用。陆军全体官兵要弘扬陆军光荣传统和优良作风，适应信息化时代陆军建设模式和运用方式的深刻变化，探索陆军发展特点和规律，按照机动作战、立体攻防的战略要求，加强顶层设计和领导管理，优化力量结构和部队编成，加快实现区域防卫型向全域作战型转变，努力建设一支强大的现代化新型陆军。\n习近平强调，火箭军是我国战略威慑的核心力量，是我国大国地位的战略支撑，是维护国家安全的重要基石。火箭军全体官兵要把握火箭军的职能定位和使命任务，按照核常兼备、全域慑战的战略要求，增强可信可靠的核威慑和核反击能力，加强中远程精确打击力量建设，增强战略制衡能力，努力建设一支强大的现代化火箭军。\n习近平强调，战略支援部队是维护国家安全的新型作战力量，是我军